## Data Loading

In [31]:
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
import time

In [32]:
twitter_path = './data/sentiment_tweets3.csv'
reddit_path = './data/depression_dataset_reddit_cleaned.csv'
mental_path = './data/mental_health.csv'

# schema = StructType([StructField('text', StringType(), True),
#                     StructField('label', IntegerType(), True)])

df_twitter = spark.read.csv(twitter_path, header=True, inferSchema=True)
df_reddit = spark.read.csv(reddit_path, header=True, inferSchema=True)
df_mental = spark.read.csv(mental_path, header=True, inferSchema=True)

In [33]:
df_twitter.printSchema()
df_reddit.printSchema()
df_mental.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- message to examine: string (nullable = true)
 |-- label (depression result): string (nullable = true)

root
 |-- clean_text: string (nullable = true)
 |-- is_depression: integer (nullable = true)

root
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



In [34]:
df_twitter = df_twitter.withColumnRenamed("message to examine", "text")
df_twitter = df_twitter.withColumnRenamed("label (depression result)", "label")
df_twitter = df_twitter.withColumn("label", df_twitter["label"].cast(IntegerType()))
df_twitter = df_twitter.drop('Index')

df_reddit = df_reddit.withColumnRenamed("clean_text", "text")
df_reddit = df_reddit.withColumnRenamed("is_depression", "label")

In [35]:
df_twitter.show()
df_reddit.show()
df_mental.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|just had a real g...|    0|
|is reading manga ...|    0|
|@comeagainjen htt...|    0|
|@lapcat Need to s...|    0|
|ADD ME ON MYSPACE...|    0|
|so sleepy. good t...|    0|
|@SilkCharm re: #n...|    0|
|23 or 24ï¿½C poss...|    0|
|nite twitterville...|    0|
|@daNanner Night, ...|    0|
|Good morning ever...|    0|
|Finally! I just c...|    0|
|kisha they cnt ge...|    0|
|@nicolerichie Yes...|    0|
|I really love ref...|    0|
|@blueaero ooo it'...|    0|
|@rokchic28 no pro...|    0|
|@shipovalov &quot...|    0|
|Once again stayed...|    0|
|@Kal_Penn I just ...|    0|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|                text|label|
+--------------------+-----+
|we understand tha...|    1|
|welcome to r depr...|    1|
|anyone else inste...|    1|
|i ve kind of stuf...|    1|
|sleep is my great...|    1|
|i m year old turn...|    1|
|i live alone and

## Data Preprocessing

In [39]:
print(df_twitter.toPandas()['text'].isnull().sum())
print(df_reddit.toPandas()['text'].isnull().sum())
print(df_mental.toPandas()['text'].isnull().sum())

print(df_twitter.toPandas()['label'].isnull().sum())
print(df_reddit.toPandas()['label'].isnull().sum())
print(df_mental.toPandas()['label'].isnull().sum())

0
0
0
9
0
0


In [41]:
df_twitter = df_twitter.dropna()

In [42]:
print(df_twitter.toPandas()['text'].isnull().sum())
print(df_reddit.toPandas()['text'].isnull().sum())
print(df_mental.toPandas()['text'].isnull().sum())

print(df_twitter.toPandas()['label'].isnull().sum())
print(df_reddit.toPandas()['label'].isnull().sum())
print(df_mental.toPandas()['label'].isnull().sum())

0
0
0
0
0
0


In [43]:
df = df_twitter.union(df_reddit).union(df_mental)

df.groupby('label').count().show()
print(df.count())

+-----+-----+
|label|count|
+-----+-----+
|    1|19974|
|    0|26039|
+-----+-----+

46013


In [44]:
trainDF, testDF = df.randomSplit([.8, .2], seed=42)
print(f"""There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set""") 

There are 37021 rows in the training set, and 8992 in the test set


## Feature Extraction

In [45]:
tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
stopwords_remover = StopWordsRemover(inputCol='tokens', outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens', outputCol='vectors')
idf = IDF(inputCol='vectors', outputCol='features')

23/04/13 10:30:22 WARN StopWordsRemover: Default locale set was [en_HK]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


## Logistic Regression

In [46]:
lr = LogisticRegression(featuresCol='features', labelCol='label')

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, vectorizer, idf, lr])

pipelineModel = pipeline.fit(trainDF)

23/04/13 10:30:24 WARN DAGScheduler: Broadcasting large task binary with size 1123.1 KiB


23/04/13 10:30:25 WARN DAGScheduler: Broadcasting large task binary with size 1124.1 KiB
23/04/13 10:30:25 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


23/04/13 10:30:26 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:26 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/13 10:30:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:28 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:28 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:28 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:28 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:30:28 WARN DAGScheduler: Broadcasting larg

In [47]:
predDF = pipelineModel.transform(testDF)

predDF.show(5)

23/04/13 10:30:46 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|label|              tokens|     filtered_tokens|             vectors|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| I'm happy and co...|    0|[, i'm, happy, an...|  [, happy, content]|(88045,[0,95,1441...|(88045,[0,95,1441...|[7.73019955190955...|[0.99956083651536...|       0.0|
| Stray Cats ~ Str...|    0|[, stray, cats, ~...|[, stray, cats, ~...|(88045,[0,902,187...|(88045,[0,902,187...|[47.5904574043106...|           [1.0,0.0]|       0.0|
| Ty - now quit re...|    0|[, ty, -, now, qu...|[, ty, -, quit, r...|(88045,[0,171,

In [48]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')

accuracy = evaluator.evaluate(predDF)
print(f'Accuracy for logistic regression: {accuracy}')

23/04/13 10:31:16 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
Accuracy for logistic regression: 0.8458629893238434


In [51]:
# Save model
model_path = "./models/lr_model"
pipelineModel.save(model_path)

AttributeError: 'PipelineModel' object has no attribute 'overwrite'

In [54]:
# Load model
from pyspark.ml import PipelineModel

loaded_pipeline_model  = PipelineModel.load("./models/lr_model")
df = pipelineModel.transform(df)

23/04/13 10:33:51 WARN StopWordsRemover: Default locale set was [en_HK]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/04/13 10:33:51 WARN StopWordsRemover: Default locale set was [en_HK]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


## Random Forest

In [55]:
rf = RandomForestClassifier(labelCol='label', maxBins=40, seed=42)

In [56]:
paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2, 4, 6])
            .addGrid(rf.numTrees, [10, 100])
            .build())

In [57]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')

In [58]:
cv = CrossValidator(estimator=rf,
                    evaluator=evaluator,
                    estimatorParamMaps=paramGrid,
                    numFolds=3,
                    parallelism=4,
                    seed=42)

pipeline = Pipeline(stages=[tokenizer, stopwords_remover, vectorizer, idf, rf])

In [59]:
start_time = time.time()

pipelineModel = pipeline.fit(trainDF)
print('Time spent:', time.time() - start_time)

23/04/13 10:34:19 WARN DAGScheduler: Broadcasting large task binary with size 1123.1 KiB


23/04/13 10:34:19 WARN DAGScheduler: Broadcasting large task binary with size 1124.1 KiB


23/04/13 10:34:20 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/04/13 10:34:20 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


23/04/13 10:34:21 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


23/04/13 10:34:31 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


...

23/04/13 10:35:03 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/04/13 10:35:04 WARN MemoryStore: Not enough space to cache rdd_1234_0 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:04 WARN MemoryStore: Not enough space to cache rdd_1234_2 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:04 WARN MemoryStore: Not enough space to cache rdd_1234_4 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:04 WARN MemoryStore: Not enough space to cache rdd_1234_3 in memory! (computed 21.8 MiB so far)


23/04/13 10:35:04 WARN MemoryStore: Not enough space to cache rdd_1234_1 in memory! (computed 91.7 MiB so far)


23/04/13 10:35:11 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/04/13 10:35:11 WARN MemoryStore: Not enough space to cache rdd_1234_1 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:11 WARN MemoryStore: Not enough space to cache rdd_1234_2 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:11 WARN MemoryStore: Not enough space to cache rdd_1234_3 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:11 WARN MemoryStore: Not enough space to cache rdd_1234_0 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:11 WARN MemoryStore: Not enough space to cache rdd_1234_4 in memory! (computed 21.8 MiB so far)


23/04/13 10:35:19 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
23/04/13 10:35:19 WARN MemoryStore: Not enough space to cache rdd_1234_2 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:19 WARN MemoryStore: Not enough space to cache rdd_1234_3 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:19 WARN MemoryStore: Not enough space to cache rdd_1234_1 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:19 WARN MemoryStore: Not enough space to cache rdd_1234_0 in memory! (computed 21.8 MiB so far)
23/04/13 10:35:19 WARN MemoryStore: Not enough space to cache rdd_1234_4 in memory! (computed 21.8 MiB so far)


Time spent: 69.99323678016663


In [60]:
predDF = pipelineModel.transform(testDF)
accuracy = evaluator.evaluate(predDF)
print(f'Accuracy for random forest: {accuracy}')

23/04/13 10:35:27 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
Accuracy for random forest: 0.7150800711743772
